# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Подготовка данных

Импортируем библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score 
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve 

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt

from sklearn.utils import shuffle

Отключим предупреждения

In [2]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

Установим случайное число

In [3]:
random_value = 12345

Читаем данные.

In [5]:
data = pd.read_csv('datasets/Churn.csv')

Рассмотрим первые 10 строк.

In [6]:
display(data.head(10))

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


Общая информация о колонках.

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [8]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Можно отметить следующие проблемы:
1. Колонка Tenure - есть пропущенные данные. Можно предположить, что пропущенными являются именно нулевые значения. (Недвижимости нет - значит можно ее и не указывать). Пометим, что нужно заполнить нолями.
2. Колонки HasCrCard, IsActiveMember, Exited - необходимо привести к типу bool.
3. Колонку Tenure можно привести к типу int.
4. От колонок RowNumber, CustomerId, Surname можно отказаться - поскольку на факт ухода клиента от этих данных не зависит, а именно это мы и исследуем в текущей работе.

Заполним пропущенные значения столбца Tenure

In [9]:
data['Tenure'] = data['Tenure'].fillna(0)

Приведем данные к необходимым типам:  
Tenure - int  
HasCrCard, IsActiveMember, Exited - bool.

In [10]:
data['Tenure'] = data['Tenure'].astype('int')

data['HasCrCard'] = data['HasCrCard'].astype('bool')
data['IsActiveMember'] = data['IsActiveMember'].astype('bool')
data['Exited'] = data['Exited'].astype('bool')

Удалим лишние колонки

In [11]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Рассмотрим соотношение значений целевого признака

In [12]:
data['Exited'].value_counts()

False    7963
True     2037
Name: Exited, dtype: int64

Соотношение примерно 1 к 4.

Заметим также, что у нас есть два категориальных столбца Gender и Geography. Модель с такими столбцами обучить мы не сможем, поэтому воспользуемся техникой прямого кодирования.

In [13]:
data = pd.get_dummies(data, drop_first = True)

Напишем функцию для определения обучающей, валидационной, кроссвалидационной и тестовой выборок из первоначального датасета

In [14]:
def super_split_data(data, target_field):
    # Получим кроссвалидационную и тестовую выборки 
    data_cv, data_test = train_test_split(data, test_size=0.2, random_state=random_value, shuffle = True, stratify = data[target_field])
    
    # Получим обучающую и валидационную выборки 
    data_train, data_valid = train_test_split(data_cv, test_size=0.25, random_state=random_value, shuffle = True, stratify = data_cv[target_field])
    
    return data_train, data_valid, data_cv, data_test

In [15]:
data_train, data_valid, data_cv, data_test = super_split_data(data, 'Exited')

Определим функции которые будут выделять отдельно признаки и отдельно целевой признак.

In [16]:
def get_target(data, field):
    return data[field]

In [17]:
def get_features(data, field):
    return data.drop([field], axis = 1)

И функцию, возвращающую отдельно признаки и целевой признак.

In [18]:
def ftr_trgt_split(data, target_field):
    features_train = get_features(data, target_field)
    target_train = get_target(data, target_field)
    return features_train, target_train

Выделим отдельно признаки и целевой признак для обучающего датасета.

In [19]:
features_train, target_train = ftr_trgt_split(data_train, 'Exited')

Выделим отдельно признаки и целевой признак для валидационного датасета.

In [20]:
features_valid, target_valid = ftr_trgt_split(data_valid, 'Exited')

Выделим отдельно признаки и целевой признак для кроссвалидационного датасета.

In [21]:
features_cv, target_cv = ftr_trgt_split(data_cv, 'Exited')

Выделим отдельно признаки и целевой признак для тестового датасета.

In [22]:
features_test, target_test = ftr_trgt_split(data_test, 'Exited')

Посмотрим на матрицу корреляции. Заметно, что коррелирующих признаков у нас нет.

In [23]:
corr = features_train.corr()
corr.style.background_gradient(cmap='coolwarm')

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
CreditScore,1.000000,-0.013553,-0.007534,0.009032,0.023945,-0.018227,0.022291,0.016233,0.010084,0.008098,-0.017697
Age,-0.013553,1.000000,-0.003559,0.035900,-0.031275,-0.018478,0.088732,0.007978,0.053099,-0.013428,-0.023332
Tenure,-0.007534,-0.003559,1.000000,-0.017577,0.025855,0.012582,-0.022474,0.000699,-0.000241,-0.001782,0.031864
Balance,0.009032,0.035900,-0.017577,1.000000,-0.292605,-0.005314,-0.004859,-0.004243,0.402504,-0.139661,0.007500
NumOfProducts,0.023945,-0.031275,0.025855,-0.292605,1.000000,0.009876,-0.002524,0.020266,-0.001764,0.009144,-0.030170
HasCrCard,-0.018227,-0.018478,0.012582,-0.005314,0.009876,1.000000,-0.009788,-0.004466,0.030326,-0.019268,-0.002305
IsActiveMember,0.022291,0.088732,-0.022474,-0.004859,-0.002524,-0.009788,1.000000,-0.004721,-0.007708,0.013719,0.033578
EstimatedSalary,0.016233,0.007978,0.000699,-0.004243,0.020266,-0.004466,-0.004721,1.000000,0.004729,-0.001947,-0.006840
Geography_Germany,0.010084,0.053099,-0.000241,0.402504,-0.001764,0.030326,-0.007708,0.004729,1.000000,-0.330374,-0.023080
Geography_Spain,0.008098,-0.013428,-0.001782,-0.139661,0.009144,-0.019268,0.013719,-0.001947,-0.330374,1.000000,0.022142


## Вывод

В ходе первичного анализа данных были обнаружены и решены следующие проблемы:

    Колонка Tenure - есть пропущенные данные. Можно предположить, что пропущенными являются именно нулевые значения. (Недвижимости нет - значит можно ее и не указывать). Пометим, что нужно заполнить нолями.
    Колонки HasCrCard, IsActiveMember, Exited - необходимо привести к типу bool.
    Колонку Tenure можно привести к типу int.
    От колонок RowNumber, CustomerId, Surname можно отказаться - поскольку на факт ухода клиента от этих данных не зависит, а именно это мы и исследуем в текущей работе.

Далее было рассмотрено соотношение значений целевого признака - 1 к 4.  
После чего исходный датасет был разбит на обучающую, валидационную, кроссвалидационную и тестовую выборки, которые в свою очередь были отдельно разделены на признаки и целевой признак. 

На обучающем датасете построили матрицу корреляции и выявили, что коррелирующих признаков у нас нет.

# 2. Исследование задачи

## Дерево решений

Попробуем обучить модели без учета дисбаланса классов.  
Начнем с модели "Дерево решений".

Переберем модели со следующими вариантами параметров:  

    макс. глубина - от 1 до 10.  
    мин. число образцов в листах - 10 значений в диапазоне от 2 до 50 с одинаковым интервалом.  
    
И оформим нахождение модели в функцию, которую дальше будем использовать на измененной обучающей выборке.

Предварительно напишем другую функцию, которая будет подбирать наилучший порог под максимальное значение F1-меры.   
Рассматриваемые значения порога от 0 до 1 с интервалом 0,1.

In [24]:
def find_best_f1(model, features, target):
    
    probabilities = model.predict_proba(features)
    probabilities_one = probabilities[:, 1]

    max_f1 = 0
    Best_Threshold = 0
    
    for threshold in np.arange(0.01, 1, 0.1):
        predicted = (probabilities_one > threshold)
        f1 = f1_score(target, predicted)
        
        if(max_f1 < f1):
            max_f1 = f1  
            Best_Threshold = threshold
    return Best_Threshold, max_f1

In [25]:
def My_DecisionTreeClassifier(features_train, target_train, features_valid, target_valid, c_weight = None):
    best_depth = None
    best_leaf = 0
    best_F1 = 0
    best_model_tree = None
    best_threshold = 0
    
    for depth in range(1, 10):
        for leaf in np.linspace(2, 50, num=10, endpoint=True, retstep=False, dtype='int'):
            model_tree = DecisionTreeClassifier(random_state = random_value, 
                                                min_samples_leaf = leaf, 
                                                max_depth = depth, 
                                                class_weight = c_weight);
            model_tree.fit(features_train, target_train) 
            #predictions = model_tree.predict(features_valid) # для однозначного определения класса
            threshold, result = find_best_f1(model_tree, features_valid, target_valid) 
            #f1_score(target_valid, predictions); # для однозначного определения класса
            if result > best_F1:
                best_depth = depth
                best_leaf = leaf
                best_F1 = result
                best_model_tree = model_tree
                best_threshold = threshold
                
    print("Порог:", best_threshold)
    print("F1-мера наилучшей модели на валидационной выборке:", best_F1)
    print("Наилучшее количество примеров в листе:", best_leaf)
    print("Наилучшая глубина дерева:", best_depth)
    
    return best_model_tree

In [26]:
%%time
best_model_tree = My_DecisionTreeClassifier(features_train, target_train, features_valid, target_valid)

Порог: 0.41000000000000003
F1-мера наилучшей модели на валидационной выборке: 0.616643929058663
Наилучшее количество примеров в листе: 7
Наилучшая глубина дерева: 7
Wall time: 3.95 s


Рассчитаем статистические параметры модели. 

Напишем специальную функцию:

In [27]:
# Используется при вероятностном определении класса
def ModelStat_Info(model, features, target):
    
    probabilities = model.predict_proba(features)
    probabilities_one = probabilities[:, 1]

    max_f1 = 0
    recall = 0
    precision = 0
    auc_roc = 0
    Best_Threshold = 0
    
    for threshold in np.arange(0.01, 1, 0.1):
        predicted = (probabilities_one > threshold)
        f1 = f1_score(target, predicted)
        
        if(max_f1 < f1):
            max_f1 = f1
            recall = recall_score(target, predicted)
            precision = precision_score(target, predicted)
            auc_roc = roc_auc_score(target, predicted)   
            Best_Threshold = threshold
    print('Порог : {}'.format(Best_Threshold))        
    print('Полнота : {}'.format(recall))
    print('Точность : {}'.format(precision))
    print('F1-мера : {}'.format(max_f1))
    print('AUC-ROC : {}'.format(auc_roc))
        

In [28]:
# Используется при однозначном определении класса
def ModelStat_Info2(model, features, target):
    predictions = model.predict(features) 
    print('Матрица ошибок : {}'
          .format(confusion_matrix(target, predictions)))

    print('Полнота : {}'.format(recall_score(target_, predictions)))
    print('Точность : {}'.format(precision_score(target, predictions)))
    print('F1-мера : {}'.format(f1_score(target, predictions)))
    
    probabilities = model.predict_proba(features)
    probabilities_one = probabilities[:, 1]
    auc_roc = roc_auc_score(target, probabilities_one)
    print('AUC-ROC : {}'.format(auc_roc))

Результаты на обучающей выборке:

In [29]:
ModelStat_Info(best_model_tree, features_train, target_train)

Порог : 0.31000000000000005
Полнота : 0.6909239574816026
Точность : 0.5913226032190343
F1-мера : 0.6372549019607844
AUC-ROC : 0.7843357488894301


Результаты на валидационной выборке:

In [30]:
ModelStat_Info(best_model_tree, features_valid, target_valid)

Порог : 0.41000000000000003
Полнота : 0.5552825552825553
Точность : 0.6932515337423313
F1-мера : 0.616643929058663
AUC-ROC : 0.7462539581183649


## Случайный лес

Переберем модели со следующими вариантами параметров:  

    Количество деревьев - от 10 до 150 с интервалом 10
    макс. глубина - от 1 до 15 с интервалом 1
    
И оформим нахождение модели в функцию, которую дальше будем использовать на измененной обучающей выборке.

In [31]:
def My_RandomForestClassifier(features_train, target_train, features_valid, target_valid, c_weight = None):
    best_fg_model = None
    best_fg_result = 0
    best_fg_est = 0
    best_fg_depth = 0
    best_threshold = 0
    
    for est in range(10, 150, 10):
        for depthh in range(1, 15, 1):
            model = RandomForestClassifier(
                random_state=random_value, 
                max_depth = depthh, 
                n_estimators=est,
                criterion = 'gini', 
                class_weight=c_weight)
            model.fit(features_train, target_train)
            
                # Используется при однозначном определении класса
            #predictions = model.predict(features_valid)
            #result = f1_score(predictions, target_valid)
            
                # Используется при вероятностном определении класса
            threshold, result = find_best_f1(model, features_valid, target_valid) 
            
            if result > best_fg_result:
                best_fg_model = model
                best_fg_est = est
                best_fg_depth = depthh
                best_fg_result = result
                best_threshold = threshold
                
                
    print("Порог:", threshold)
    print("F1-мера наилучшей модели на валидационной выборке:", best_fg_result)
    print("Наилучшее количество деревьев:", best_fg_est)
    print("Наилучшая глубина дерева:", best_fg_depth)
  #  print("Минимальное количество примеров для разделения узла:", best_min_samples_split)
    
    return best_fg_model

In [32]:
%%time
best_fg_model = My_RandomForestClassifier(features_train, target_train, features_valid, target_valid)

Порог: 0.31000000000000005
F1-мера наилучшей модели на валидационной выборке: 0.6360153256704981
Наилучшее количество деревьев: 130
Наилучшая глубина дерева: 8
Wall time: 1min 40s


Результаты на обучающей выборке:

In [33]:
ModelStat_Info(best_fg_model, features_train, target_train)

Порог : 0.31000000000000005
Полнота : 0.6590351594439902
Точность : 0.7367458866544789
F1-мера : 0.6957272334915839
AUC-ROC : 0.7993731376035108


Результаты на валидационной выборке: 

In [34]:
ModelStat_Info(best_fg_model, features_valid, target_valid)

Порог : 0.31000000000000005
Полнота : 0.6117936117936118
Точность : 0.6622340425531915
F1-мера : 0.6360153256704981
AUC-ROC : 0.7660349101027067


## Логистическая регрессия

In [35]:
def My_LogisticRegression(features_train, target_train, features_valid, target_valid, c_weight = None):
    model_logRegression = LogisticRegression(random_state = random_value, solver='liblinear', class_weight=c_weight)
    model_logRegression.fit(features_train, target_train)
        
        # Используется при однозначном определении класса
    #predictions = model_logRegression.predict(features_valid)
        
        # Используется при вероятностном определении класса
    threshold, result = find_best_f1(model_logRegression, features_valid, target_valid) 
    
    print("Порог:", threshold)
    print('F1-мера : {}'.format(result))
    
    return model_logRegression

In [36]:
model_logRegression = My_LogisticRegression(features_train, target_train, features_valid, target_valid)

Порог: 0.21000000000000002
F1-мера : 0.3866459627329193


In [37]:
ModelStat_Info(model_logRegression, features_valid, target_valid)

Порог : 0.21000000000000002
Полнота : 0.6117936117936118
Точность : 0.282633371169126
F1-мера : 0.3866459627329193
AUC-ROC : 0.6075289465119974


## Вывод:

Обучили модели без учета дисбаланса классов.
##### "Дерево решений".

Перебрали модели со следующими вариантами параметров:

макс. глубина - от 1 до 10.  
мин. число образцов в листах - 10 значений в диапазоне от 2 до 50 с одинаковым интервалом.  

Оформили нахождение модели в функцию, которую дальше будем использовать на измененной обучающей выборке.
Написали также функцию, которая будет подбирать наилучший порог под максимальное значение F1-меры.
Рассматриваемые значения порога от 0.01 до 1 с интервалом 0,1.

###### Лучшая модель (без учета дисбаланса классов):
Порог: 0.41
Наилучшее количество примеров в листе: 7
Наилучшая глубина дерева: 7
F1-мера наилучшей модели на валидационной выборке: 0.62
AUC-ROC : 0.78

##### Случайный лес
Перебрали модели со следующими вариантами параметров:

Количество деревьев - от 10 до 150 с интервалом 10
макс. глубина - от 1 до 15 с интервалом 1

###### Лучшая модель (без учета дисбаланса классов):
Порог: 0.31
Наилучшее количество деревьев: 130
Наилучшая глубина дерева: 8
F1-мера наилучшей модели на валидационной выборке: 0.64
AUC-ROC : 0.77

##### Логистическая регрессия
Построили модель с параметром solver='liblinear'  
      
###### Лучшая модель (без учета дисбаланса классов):
Порог : 0.21
F1-мера : 0.39
AUC-ROC : 0.61

##### Обобщение:
Благодаря поиску оптимального порога мы получили довольно неплохие предварительные модели (даже без учета дисбаланса классов). Если бы порог был стандартным - результаты были бы несколько хуже.

# Борьба с дисбалансом

## Downsampling

Напишем функцию уменьшения выборки:

In [38]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=random_value)] + [features_ones]) 
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=random_value)] + [target_ones]) 
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=random_value)
    
    return features_downsampled, target_downsampled

Поскольку соотношение классов, посчитанное нами ранее, равно 1 к 4. Можно взять параметр Fraction = 0.3.

In [39]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.3)

### Downsampling. Дерево решений

Найдем теперь лучшую модель "дерева решений" на новых данных. И посмотрим её правильность. 

In [40]:
%%time
best_model_tree_Downsampling = My_DecisionTreeClassifier(features_downsampled, target_downsampled, features_valid, target_valid)

Порог: 0.51
F1-мера наилучшей модели на валидационной выборке: 0.598669623059867
Наилучшее количество примеров в листе: 28
Наилучшая глубина дерева: 5
Wall time: 2.74 s


In [41]:
ModelStat_Info(best_model_tree_Downsampling, features_valid, target_valid)

Порог : 0.51
Полнота : 0.6633906633906634
Точность : 0.5454545454545454
F1-мера : 0.598669623059867
AUC-ROC : 0.761073862768778


### Downsampling. Случайный лес

In [42]:
%%time
best_fg_model_Downsampling = My_RandomForestClassifier(features_downsampled, target_downsampled, features_valid, target_valid)

Порог: 0.51
F1-мера наилучшей модели на валидационной выборке: 0.6436285097192225
Наилучшее количество деревьев: 140
Наилучшая глубина дерева: 12
Wall time: 1min 7s


In [43]:
ModelStat_Info(best_fg_model_Downsampling, features_valid, target_valid)

Порог : 0.51
Полнота : 0.7321867321867321
Точность : 0.5741811175337187
F1-мера : 0.6436285097192225
AUC-ROC : 0.796727389947729


### Downsampling. Логистическая регрессия

In [44]:
%%time
model_logRegression_Downsampling = My_LogisticRegression(features_downsampled, target_downsampled, features_valid, target_valid)

Порог: 0.51
F1-мера : 0.45837414299706175
Wall time: 40 ms


In [45]:
ModelStat_Info(model_logRegression_Downsampling, features_valid, target_valid)

Порог : 0.51
Полнота : 0.5749385749385749
Точность : 0.3811074918566775
F1-мера : 0.45837414299706175
AUC-ROC : 0.668197473282219


## 3. Upsampling

Напишем функцию увеличения выборки:

In [46]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=random_value)
    
    return features_upsampled, target_upsampled

In [47]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 3)

### Upsampling. Дерево решений

In [48]:
%%time
best_model_tree_Upsampling = My_DecisionTreeClassifier(features_upsampled, target_upsampled, features_valid, target_valid)

Порог: 0.6100000000000001
F1-мера наилучшей модели на валидационной выборке: 0.609442060085837
Наилучшее количество примеров в листе: 28
Наилучшая глубина дерева: 6
Wall time: 4.67 s


In [49]:
ModelStat_Info(best_model_tree_Upsampling, features_valid, target_valid)

Порог : 0.6100000000000001
Полнота : 0.6977886977886978
Точность : 0.540952380952381
F1-мера : 0.609442060085837
AUC-ROC : 0.7732509088441291


### Upsampling. Случайный лес

In [50]:
%%time
best_fg_model_Upsampling = My_RandomForestClassifier(features_upsampled, target_upsampled, features_valid, target_valid)

Порог: 0.41000000000000003
F1-мера наилучшей модели на валидационной выборке: 0.6495327102803738
Наилучшее количество деревьев: 60
Наилучшая глубина дерева: 8
Wall time: 2min 13s


In [51]:
ModelStat_Info(best_fg_model_Upsampling, features_valid, target_valid)

Порог : 0.51
Полнота : 0.683046683046683
Точность : 0.6191536748329621
F1-мера : 0.6495327102803738
AUC-ROC : 0.7878510251391607


### Upsampling. Логистическая регрессия

In [52]:
%%time
model_logRegression_Upsampling = My_LogisticRegression(features_upsampled, target_upsampled, features_valid, target_valid)

Порог: 0.41000000000000003
F1-мера : 0.44509516837481694
Wall time: 70 ms


In [53]:
ModelStat_Info(model_logRegression_Upsampling, features_valid, target_valid)

Порог : 0.41000000000000003
Полнота : 0.7469287469287469
Точность : 0.3169968717413973
F1-мера : 0.44509516837481694
AUC-ROC : 0.6678774305892949


## Параметр Class_weight = 'balanced' (CW)

### CW. Дерево решений

In [54]:
%%time
best_model_tree_CW = My_DecisionTreeClassifier(features_train, target_train, features_valid, target_valid, 'balanced')

Порог: 0.6100000000000001
F1-мера наилучшей модели на валидационной выборке: 0.6098081023454158
Наилучшее количество примеров в листе: 18
Наилучшая глубина дерева: 6
Wall time: 4.11 s


In [55]:
ModelStat_Info(best_model_tree_CW, features_valid, target_valid)

Порог : 0.6100000000000001
Полнота : 0.7027027027027027
Точность : 0.5386064030131826
F1-мера : 0.6098081023454158
AUC-ROC : 0.7744524185202151


### CW. Случайный лес

In [56]:
%%time
best_fg_model_CW = My_RandomForestClassifier(features_train, 
                                                     target_train, 
                                                     features_valid, 
                                                     target_valid, 
                                                     'balanced')

Порог: 0.41000000000000003
F1-мера наилучшей модели на валидационной выборке: 0.64075382803298
Наилучшее количество деревьев: 100
Наилучшая глубина дерева: 9
Wall time: 1min 44s


In [57]:
ModelStat_Info(best_fg_model_CW, features_valid, target_valid)

Порог : 0.51
Полнота : 0.6683046683046683
Точность : 0.6153846153846154
F1-мера : 0.64075382803298
AUC-ROC : 0.7807938909633825


### CW. Логистическая регрессия

In [58]:
%%time
model_logRegression_CW = My_LogisticRegression(features_train, target_train, features_valid, target_valid, 'balanced')

Порог: 0.51
F1-мера : 0.4829351535836178
Wall time: 60 ms


In [59]:
ModelStat_Info(model_logRegression_CW, features_valid, target_valid)

Порог : 0.51
Полнота : 0.6953316953316954
Точность : 0.36993464052287583
F1-мера : 0.4829351535836178
AUC-ROC : 0.6963789675654082


## Вывод.

Произвели три попытки борьбы с дисбалансом классов. Получились следующие результаты (в скобках указаны значения без учета дисбаланса): 


##### Дерево решений:

###### Downsampling - удаление части выборки преобладающего класса  
Наилучшая модель:
    Порог: 0.51     
    Наилучшее количество примеров в листе: 28  
    Наилучшая глубина дерева: 5
    F1-мера наилучшей модели на валидационной выборке: 0.599 (0.62)
    AUC-ROC : 0.76 (0.78)

###### Upsampling - дублирование выборки класса в меньшинстве  
Наилучшая модель:
    Порог: 0.61
    Наилучшее количество примеров в листе: 28
    Наилучшая глубина дерева: 6
    F1-мера наилучшей модели на валидационной выборке: 0.61 (0.62)
    AUC-ROC : 0.77 (0.78)

###### C помощью параметра Class_weight = 'balanced'.
Наилучшая модель:
    Порог: 0.61
    Наилучшее количество примеров в листе: 18
    Наилучшая глубина дерева: 6
    F1-мера наилучшей модели на валидационной выборке: 0.61 (0.62)
    AUC-ROC : 0.77 (0.78)


##### Случайный лес:

###### Downsampling - удаление части выборки преобладающего класса  
Наилучшая модель:
    Порог: 0.51
    Наилучшее количество деревьев: 140
    Наилучшая глубина дерева: 12
    F1-мера наилучшей модели на валидационной выборке: 0.64 (0.64)
    AUC-ROC : 0.78 (0.77)

###### Upsampling - дублирование выборки класса в меньшинстве  
Наилучшая модель:
    Порог: 0.41
    Наилучшее количество деревьев: 60
    Наилучшая глубина дерева: 8
    F1-мера наилучшей модели на валидационной выборке: 0.65 (0.64)
    AUC-ROC : 0.79 (0.77)
    
###### C помощью параметра Class_weight = 'balanced'.
Наилучшая модель:
    Порог: 0.31
    Наилучшее количество деревьев: 60
    Наилучшая глубина дерева: 9
    F1-мера наилучшей модели на валидационной выборке: 0.65 (0.64)
    AUC-ROC : 0.79 (0.77)
    
##### Логистическая регрессия: 

###### Downsampling - удаление части выборки преобладающего класса  
Наилучшая модель:
    Порог : 0.51
    F1-мера : 0.46 (0.39)
    AUC-ROC : 0.67 (0.61)
    
###### Upsampling - дублирование выборки класса в меньшинстве  
Наилучшая модель:
    Порог : 0.51
    F1-мера : 0.46 (0.39)
    AUC-ROC : 0.67 (0.61)

###### C помощью параметра Class_weight = 'balanced'.
Наилучшая модель:
    Порог : 0.61
    F1-мера : 0.52 (0.39)
    AUC-ROC : 0.71 (0.61)
    
##### Обобщая можно сделать следующие выводы: 
1.    Значение F1-меры для логистической регрессии очень сильно зависит от балансировки весов класса. Веса используются в формулах вычисления принадлежности (прогнозирования) классу.  
2.    На модели "Дерево решений" и  "случайный лес" учёт баланса классов влияет не очень сильно (за исключением Downsampling - лишает алгоритм данных)
3.    Для логистической регрессии параметр Class_weight = 'balanced' работает эффективнее.  
4.    Приблизительно одинаковые значения AUC-ROC для моделей "Дерево решений" и  "случайный лес" подтверждают, что баланс классов на их результативность не влияет. В отличии от модели "Логистическая регрессия". Это связано с особенностями алгоритмов данных моделей.

# 4. Тестирование модели

Протестируем некоторые из полученных моделей.

Напишем для этого специальную функцию для определения F1-меры по установленному ранее порогу.

In [60]:
# Используется при вероятностном определении класса
def ModelStat_Info_threshold(model, features, target, threshold):
    
    probabilities = model.predict_proba(features)
    probabilities_one = probabilities[:, 1]

    max_f1 = 0
    recall = 0
    precision = 0
    auc_roc = 0
    Best_Threshold = 0
    
    predicted = (probabilities_one > threshold)
    f1 = f1_score(target, predicted)
    recall = recall_score(target, predicted)
    precision = precision_score(target, predicted)
    auc_roc = roc_auc_score(target, predicted)
    
    print('Порог : {}'.format(threshold))        
    print('Полнота : {}'.format(recall))
    print('Точность : {}'.format(precision))
    print('F1-мера : {}'.format(f1))
    print('AUC-ROC : {}'.format(auc_roc))
        

## Дерево решений 

Модель "Дерево решений" с параметром Class_weight = 'balanced'. Наилучший порог, рассчитанный ранее, равен 0.61

In [61]:
%%time
best_model_tree_CW.fit(features_cv, target_cv)
ModelStat_Info_threshold(best_model_tree_CW, features_test, target_test, 0.61)

Порог : 0.61
Полнота : 0.7100737100737101
Точность : 0.5017361111111112
F1-мера : 0.5879959308240083
AUC-ROC : 0.7649552480060955
Wall time: 52 ms


## Случайный лес

Модель "Случайный лес" не учитывающая дисбаланс классов. Наилучший порог, рассчитанный ранее, равен 0.31

In [62]:
best_fg_model.fit(features_cv, target_cv)
ModelStat_Info_threshold(best_fg_model, features_test, target_test, 0.31)

Порог : 0.31
Полнота : 0.6314496314496314
Точность : 0.6539440203562341
F1-мера : 0.6425
AUC-ROC : 0.7730380611736544


Модель "Случайный лес" Downsampling. Наилучший порог, рассчитанный ранее, равен 0.51

In [63]:
best_fg_model_Downsampling.fit(features_cv, target_cv)
ModelStat_Info_threshold(best_fg_model_Downsampling, features_test, target_test, 0.51)

Порог : 0.51
Полнота : 0.4398034398034398
Точность : 0.788546255506608
F1-мера : 0.5646687697160884
AUC-ROC : 0.7048358065307218


Модель "Случайный лес" Upsampling. Наилучший порог, рассчитанный ранее, равен 0.41

In [64]:
best_fg_model_Upsampling.fit(features_cv, target_cv)
ModelStat_Info_threshold(best_fg_model_Upsampling, features_test, target_test, 0.41)

Порог : 0.41
Полнота : 0.4914004914004914
Точность : 0.7490636704119851
F1-мера : 0.5934718100890207
AUC-ROC : 0.7246707416198942


Модель "Случайный лес" с параметром Class_weight = 'balanced'. Наилучший порог, рассчитанный ранее, равен 0.31

In [65]:
best_fg_model_CW.fit(features_cv, target_cv)
ModelStat_Info_threshold(best_fg_model_CW, features_test, target_test, 0.31)

Порог : 0.31
Полнота : 0.8845208845208845
Точность : 0.3866809881847476
F1-мера : 0.5381165919282511
AUC-ROC : 0.7630388477846106


## Логистическая регрессия

Модель "Логистическая регрессия" с параметром Class_weight = 'balanced'

In [66]:
%%time
model_logRegression_CW.fit(features_cv, target_cv)
ModelStat_Info(model_logRegression_CW, features_test, target_test)

Порог : 0.51
Полнота : 0.6904176904176904
Точность : 0.3561470215462611
F1-мера : 0.4698996655518395
AUC-ROC : 0.6857612620324485
Wall time: 90 ms


## Вывод

Переобучили наилучшие модели, полученные ранее, на объединенной выборке (обуч. + валид.) и посмотрели результаты.

Выводы получились довольно неожиданными: (в скобках значения на валидационной выборке)

    Дерево решений. CW:
        F1-мера : 0.59 (0.61)
    
    Модель "Случайный лес" не учитывающая дисбаланс классов:
        F1-мера : 0.64 (0.64)
        
    Модель "Случайный лес". Downsampling:
        F1-мера : 0.56 (0.64)
    
    Модель "Случайный лес". Upsampling:
        F1-мера : 0.59 (0.65)

    Модель "Случайный лес". CW:
        F1-мера : 0.53 (0.65)
        
    Модель "Логистическая регрессия":
        F1-мера : 0.50 (0.52)
        
    Учитывать баланс классов стоит только в модели логистическая регрессия.
    В модели "дерево решений" и "случайный лес" он, будто, вносит некоторый хаос. Результаты на тестовой выборке для моделей, учитывающих дисбаланс классов, заметно просели. Есть ощущение, что с учетом этого параметра данные модели переобучаются. 
    
    Особенно на фоне модели "Случайный лес" без учета дисбаланса классов - эта модель показала себя стабильно. На валидационной выборке значение F1-меры - 0,64 и на тестовой те же 0,64. 

    В итоге - лучшая модель - "Случайный лес" без учета дисбаланса классов с параметрами:
        количество деревьев: 130 
        глубина дерева: 8
    
    А учёт баланса классов - обязателен для моделей логистической регрессии.

# Кросс-валидация (дополнительно)

Построим модель случайного леса с помощью кросс-валидации со следующим перебором параметров:  

    количество деревьев - от 10 до 151 с шагом 15.  
    максимальная глубина - от 1 до 20 с шагом 2.
    


In [67]:
%%time
param_dist = { 'n_estimators': range (10, 151, 15),
    'max_depth': range (1, 20, 2)}

model_forestCV = RandomForestClassifier(random_state = random_value)

rs_CV = GridSearchCV(model_forestCV, 
                     param_grid=param_dist, 
                     cv = 4,
                     return_train_score = True,
                     n_jobs = 1,
                     verbose = 2
                    )

Wall time: 0 ns


In [68]:
%%time
rs_CV.fit(features_cv, target_cv)

Fitting 4 folds for each of 100 candidates, totalling 400 fits
[CV] max_depth=1, n_estimators=10 ....................................
[CV] ..................... max_depth=1, n_estimators=10, total=   0.1s
[CV] max_depth=1, n_estimators=10 ....................................
[CV] ..................... max_depth=1, n_estimators=10, total=   0.1s
[CV] max_depth=1, n_estimators=10 ....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ..................... max_depth=1, n_estimators=10, total=   0.1s
[CV] max_depth=1, n_estimators=10 ....................................
[CV] ..................... max_depth=1, n_estimators=10, total=   0.1s
[CV] max_depth=1, n_estimators=25 ....................................
[CV] ..................... max_depth=1, n_estimators=25, total=   0.1s
[CV] max_depth=1, n_estimators=25 ....................................
[CV] ..................... max_depth=1, n_estimators=25, total=   0.1s
[CV] max_depth=1, n_estimators=25 ....................................
[CV] ..................... max_depth=1, n_estimators=25, total=   0.1s
[CV] max_depth=1, n_estimators=25 ....................................
[CV] ..................... max_depth=1, n_estimators=25, total=   0.1s
[CV] max_depth=1, n_estimators=40 ....................................
[CV] ..................... max_depth=1, n_estimators=40, total=   0.2s
[CV] max_depth=1, n_estimators=40 ....................................
[CV] .

[CV] ..................... max_depth=3, n_estimators=85, total=   0.4s
[CV] max_depth=3, n_estimators=85 ....................................
[CV] ..................... max_depth=3, n_estimators=85, total=   0.4s
[CV] max_depth=3, n_estimators=85 ....................................
[CV] ..................... max_depth=3, n_estimators=85, total=   0.4s
[CV] max_depth=3, n_estimators=85 ....................................
[CV] ..................... max_depth=3, n_estimators=85, total=   0.4s
[CV] max_depth=3, n_estimators=100 ...................................
[CV] .................... max_depth=3, n_estimators=100, total=   0.5s
[CV] max_depth=3, n_estimators=100 ...................................
[CV] .................... max_depth=3, n_estimators=100, total=   0.5s
[CV] max_depth=3, n_estimators=100 ...................................
[CV] .................... max_depth=3, n_estimators=100, total=   0.6s
[CV] max_depth=3, n_estimators=100 ...................................
[CV] .

[CV] .................... max_depth=5, n_estimators=145, total=   0.8s
[CV] max_depth=5, n_estimators=145 ...................................
[CV] .................... max_depth=5, n_estimators=145, total=   0.8s
[CV] max_depth=7, n_estimators=10 ....................................
[CV] ..................... max_depth=7, n_estimators=10, total=   0.1s
[CV] max_depth=7, n_estimators=10 ....................................
[CV] ..................... max_depth=7, n_estimators=10, total=   0.1s
[CV] max_depth=7, n_estimators=10 ....................................
[CV] ..................... max_depth=7, n_estimators=10, total=   0.1s
[CV] max_depth=7, n_estimators=10 ....................................
[CV] ..................... max_depth=7, n_estimators=10, total=   0.1s
[CV] max_depth=7, n_estimators=25 ....................................
[CV] ..................... max_depth=7, n_estimators=25, total=   0.2s
[CV] max_depth=7, n_estimators=25 ....................................
[CV] .

...

[CV] .................... max_depth=15, n_estimators=55, total=   0.6s
[CV] max_depth=15, n_estimators=55 ...................................
[CV] .................... max_depth=15, n_estimators=55, total=   0.7s
[CV] max_depth=15, n_estimators=55 ...................................
[CV] .................... max_depth=15, n_estimators=55, total=   0.8s
[CV] max_depth=15, n_estimators=55 ...................................
[CV] .................... max_depth=15, n_estimators=55, total=   0.6s
[CV] max_depth=15, n_estimators=70 ...................................
[CV] .................... max_depth=15, n_estimators=70, total=   0.7s
[CV] max_depth=15, n_estimators=70 ...................................
[CV] .................... max_depth=15, n_estimators=70, total=   0.7s
[CV] max_depth=15, n_estimators=70 ...................................
[CV] .................... max_depth=15, n_estimators=70, total=   0.7s
[CV] max_depth=15, n_estimators=70 ...................................
[CV] .

[CV] ................... max_depth=17, n_estimators=115, total=   1.2s
[CV] max_depth=17, n_estimators=115 ..................................
[CV] ................... max_depth=17, n_estimators=115, total=   1.2s
[CV] max_depth=17, n_estimators=130 ..................................
[CV] ................... max_depth=17, n_estimators=130, total=   1.3s
[CV] max_depth=17, n_estimators=130 ..................................
[CV] ................... max_depth=17, n_estimators=130, total=   1.2s
[CV] max_depth=17, n_estimators=130 ..................................
[CV] ................... max_depth=17, n_estimators=130, total=   1.2s
[CV] max_depth=17, n_estimators=130 ..................................
[CV] ................... max_depth=17, n_estimators=130, total=   1.2s
[CV] max_depth=17, n_estimators=145 ..................................
[CV] ................... max_depth=17, n_estimators=145, total=   1.4s
[CV] max_depth=17, n_estimators=145 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  4.6min finished


Wall time: 4min 34s


GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=12345),
             n_jobs=1,
             param_grid={'max_depth': range(1, 20, 2),
                         'n_estimators': range(10, 151, 15)},
             return_train_score=True, verbose=2)

Посмотрим какие получились параметры наилучшей модели.

In [69]:
rs_CV.best_params_

{'max_depth': 11, 'n_estimators': 85}

Проверим результаты:

In [70]:
model_forestCV.fit(features_cv, target_cv) 
ModelStat_Info(model_forestCV, features_test, target_test)

Порог : 0.31000000000000005
Полнота : 0.6707616707616708
Точность : 0.5909090909090909
F1-мера : 0.6283084004602993
AUC-ROC : 0.7760588014825303


## Вывод
    
Дополнительно построили модель с помощью кросс-валидации.  
Получили наилучшие параметры для модели:  
    Количество деревьев - 145  
    Максимальная глубина - 11
    
В итоге получились следующие результаты:

    Порог : 0.31
    Полнота : 0.62
    Точность : 0.6
    F1-мера : 0.61
    AUC-ROC : 0.76

# Вывод

## Этап предобработки данных
В ходе первичного анализа данных были обнаружены и решены следующие проблемы:

    Колонка Tenure - есть пропущенные данные. Можно предположить, что пропущенными являются именно нулевые значения. (Недвижимости нет - значит можно ее и не указывать). Пометим, что нужно заполнить нолями.
    Колонки HasCrCard, IsActiveMember, Exited - необходимо привести к типу bool.
    Колонку Tenure можно привести к типу int.
    От колонок RowNumber, CustomerId, Surname можно отказаться - поскольку на факт ухода клиента от этих данных не зависит, а именно это мы и исследуем в текущей работе.

Далее было рассмотрено соотношение значений целевого признака - 1 к 4.  
После чего исходный датасет был разбит на обучающую, валидационную, кроссвалидационную и тестовую выборки, которые в свою очередь были отдельно разделены на признаки и целевой признак. 

На обучающем датасете построили матрицу корреляции и выявили, что коррелирующих признаков у нас нет.

## Обучение моделей без учета дисбаланса классов

Обучили модели без учета дисбаланса классов.
##### "Дерево решений".

Перебрали модели со следующими вариантами параметров:

макс. глубина - от 1 до 10.  
мин. число образцов в листах - 10 значений в диапазоне от 2 до 50 с одинаковым интервалом.  

Оформили нахождение модели в функцию, которую дальше будем использовать на измененной обучающей выборке.
Написали также функцию, которая будет подбирать наилучший порог под максимальное значение F1-меры.
Рассматриваемые значения порога от 0.01 до 1 с интервалом 0,1.

###### Лучшая модель (без учета дисбаланса классов):
Порог: 0.41  
Наилучшее количество примеров в листе: 7  
Наилучшая глубина дерева: 7   
F1-мера наилучшей модели на валидационной выборке: 0.62  
AUC-ROC : 0.78  

##### Случайный лес
Перебрали модели со следующими вариантами параметров:

Количество деревьев - от 10 до 150 с интервалом 10  
макс. глубина - от 1 до 15 с интервалом 1  

###### Лучшая модель (без учета дисбаланса классов):
Порог: 0.31  
Наилучшее количество деревьев: 130  
Наилучшая глубина дерева: 8   
F1-мера наилучшей модели на валидационной выборке: 0.64  
AUC-ROC : 0.77  

##### Логистическая регрессия
Построили модель с параметром solver='liblinear'  
      
###### Лучшая модель (без учета дисбаланса классов):
Порог : 0.21
F1-мера : 0.39
AUC-ROC : 0.61

##### Обобщение:
Благодаря поиску оптимального порога мы получили довольно неплохие предварительные модели (даже без учета дисбаланса классов). Если бы порог был стандартным - результаты были бы несколько хуже.

## Обучение моделей с учетом дисбаланса классов

Произвели три попытки борьбы с дисбалансом классов. Получились следующие результаты (в скобках указаны значения без учета дисбаланса): 


##### Дерево решений:

###### Downsampling - удаление части выборки преобладающего класса  
Наилучшая модель:

    Порог: 0.51     
    Наилучшее количество примеров в листе: 28  
    Наилучшая глубина дерева: 5
    F1-мера наилучшей модели на валидационной выборке: 0.599 (0.62)
    AUC-ROC : 0.76 (0.78)

###### Upsampling - дублирование выборки класса в меньшинстве  
Наилучшая модель:

    Порог: 0.61
    Наилучшее количество примеров в листе: 28
    Наилучшая глубина дерева: 6
    F1-мера наилучшей модели на валидационной выборке: 0.61 (0.62)
    AUC-ROC : 0.77 (0.78)

###### C помощью параметра Class_weight = 'balanced'.
Наилучшая модель:

    Порог: 0.61
    Наилучшее количество примеров в листе: 18
    Наилучшая глубина дерева: 6
    F1-мера наилучшей модели на валидационной выборке: 0.61 (0.62)
    AUC-ROC : 0.77 (0.78)


##### Случайный лес:

###### Downsampling - удаление части выборки преобладающего класса  
Наилучшая модель:

    Порог: 0.51
    Наилучшее количество деревьев: 140
    Наилучшая глубина дерева: 12
    F1-мера наилучшей модели на валидационной выборке: 0.64 (0.64)
    AUC-ROC : 0.78 (0.77)

###### Upsampling - дублирование выборки класса в меньшинстве  
Наилучшая модель:

    Порог: 0.41
    Наилучшее количество деревьев: 60
    Наилучшая глубина дерева: 8
    F1-мера наилучшей модели на валидационной выборке: 0.65 (0.64)
    AUC-ROC : 0.79 (0.77)
    
###### C помощью параметра Class_weight = 'balanced'.
Наилучшая модель:

    Порог: 0.31
    Наилучшее количество деревьев: 60
    Наилучшая глубина дерева: 9
    F1-мера наилучшей модели на валидационной выборке: 0.65 (0.64)
    AUC-ROC : 0.79 (0.77)
    
##### Логистическая регрессия: 

###### Downsampling - удаление части выборки преобладающего класса  
Наилучшая модель:

    Порог : 0.51
    F1-мера : 0.46 (0.39)
    AUC-ROC : 0.67 (0.61)
    
###### Upsampling - дублирование выборки класса в меньшинстве  
Наилучшая модель:

    Порог : 0.51
    F1-мера : 0.46 (0.39)
    AUC-ROC : 0.67 (0.61)

###### C помощью параметра Class_weight = 'balanced'.
Наилучшая модель:

    Порог : 0.61
    F1-мера : 0.52 (0.39)
    AUC-ROC : 0.71 (0.61)
    
##### Обобщая можно сделать следующие выводы: 
1.    Значение F1-меры для логистической регрессии очень сильно зависит от балансировки весов класса. Веса используются в формулах вычисления принадлежности (прогнозирования) классу.  
2.    На модели "Дерево решений" и  "случайный лес" учёт баланса классов влияет не очень сильно (за исключением Downsampling - лишает алгоритм данных)
3.    Для логистической регрессии параметр Class_weight = 'balanced' работает эффективнее.  
4.    Приблизительно одинаковые значения AUC-ROC для моделей "Дерево решений" и  "случайный лес" подтверждают, что баланс классов на их результативность не влияет. В отличии от модели "Логистическая регрессия". Это связано с особенностями алгоритмов данных моделей.

## Тестирование

Переобучили наилучшие модели, полученные ранее, на объединенной выборке (обуч. + валид.) и посмотрели результаты.

Выводы получились довольно неожиданными: (в скобках значения на валидационной выборке)

    Дерево решений. CW:
        F1-мера : 0.59 (0.61)
    
    Модель "Случайный лес" не учитывающая дисбаланс классов:
        F1-мера : 0.64 (0.64)
        
    Модель "Случайный лес". Downsampling:
        F1-мера : 0.56 (0.64)
    
    Модель "Случайный лес". Upsampling:
        F1-мера : 0.59 (0.65)

    Модель "Случайный лес". CW:
        F1-мера : 0.53 (0.65)
        
    Модель "Логистическая регрессия":
        F1-мера : 0.50 (0.52)
        
    Учитывать баланс классов стоит только в модели логистическая регрессия.
    В модели "дерево решений" и "случайный лес" он, будто, вносит некоторый хаос. Результаты на тестовой выборке для моделей, учитывающих дисбаланс классов, заметно просели. Есть ощущение, что с учетом этого параметра данные модели переобучаются. 
    
    Особенно на фоне модели "Случайный лес" без учета дисбаланса классов - эта модель показала себя стабильно. На валидационной выборке значение F1-меры - 0,64 и на тестовой те же 0,64. 

###### В итоге - лучшая модель - "Случайный лес" без учета дисбаланса классов с параметрами:
        количество деревьев: 130 
        глубина дерева: 8
    
###### А учёт баланса классов - обязателен для моделей логистической регрессии.
    
## Кросс-валидация   

Дополнительно построили модель случайного леса с помощью кросс-валидации со следующим перебором параметров:  

количество деревьев - от 10 до 151 с шагом 15.   
максимальная глубина - от 1 до 20 с шагом 2.  

Получили наилучшие параметры для модели:  
Количество деревьев - 145  
Максимальная глубина - 11  

В итоге получились следующие результаты:  
Порог : 0.31  
Полнота : 0.62  
Точность : 0.6  
F1-мера : 0.61  
AUC-ROC : 0.76  